In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
# Use .item() to convert from numpy array to dictionary for easier loading
track_data = np.load('tracks.npy', allow_pickle=True).item()
track_df = pd.DataFrame.from_records(track_data)

crthit_data = np.load('crthits.npy', allow_pickle=True).item()
crthit_df = pd.DataFrame.from_records(crthit_data)

match_candidate_data = np.load('match_candidates.npy', allow_pickle=True).item()
match_candidate_df = pd.DataFrame.from_records(match_candidate_data)

crt_plane_df = pd.read_csv('data/crt_geometry.csv')

In [ ]:
match_candidate_df.head()

In [ ]:
# Split track and CRT hit dataframes into matched and unmatched dataframes. 
# This will make it easier to give (un)matched objects different colors, markers, etc

# Select a subset of Tracks (and CRT hits) to plot, or uncomment the line below this to plot all objects
matched_track_ids = [7, 13]
#matched_track_ids = match_candidate_df['trackID'].values # Plot all tracks and CRT hits

# Filter the rows where the trackID column matches any value in the matched_track_ids list
matched_objects_df = match_candidate_df[match_candidate_df['trackID'].isin(matched_track_ids)]

# Create a dictionary where the keys are trackIDs and values are crthitIDs
track_crthit_dict = matched_objects_df.set_index('trackID')['crthitID'].to_dict()

# Use that dictionary to split tracks and CRT hits into "matched" and "unmatched" dataframes
matched_track_ids  = track_crthit_dict.keys()
matched_track_df = track_df[track_df['id'].isin(matched_track_ids)]
unmatched_track_df = track_df.drop(matched_track_df.index)

matched_crthit_ids = track_crthit_dict.values()
matched_crthit_df = crthit_df[crthit_df['id'].isin(matched_crthit_ids)]
unmatched_crthit_df = crthit_df.drop(matched_crthit_df.index)

In [ ]:
def plot_crt_tpc_matches(matched_track_df, unmatched_track_df, 
                         matched_crthit_df, unmatched_crthit_df, 
                         crt_plane_df,
                         track_ids=[], plot_unmatched_objects=True):
    
    # If track_ids is supplied, only plot that subset
    matches_only = False
    if track_ids:
        matches_only = True
    
    #if track_ids:
    #    matched_track_df_mask = matched_track_df['id'].isin(track_ids)
    #    matched_track_df = matched_track_df[matched_track_df_mask]
    
    traces = []

    ##### TRACK TRACES #####
    for index, track in matched_track_df.iterrows():

        track_points = track['points']
        track_id = track['id']

        track_xs = track_points[:,0]
        track_ys = track_points[:,1]
        track_zs = track_points[:,2]

        matched_track_trace = go.Scatter3d(
            x = track_xs,
            # Flip y and z so that plotly plots the long axis as the z-direction
            y = track_zs,
            z = track_ys,
            mode='markers',
            marker=dict(size=1, color='red'),
            name='Track ID ' + str(track_id),
            customdata=np.stack((track_xs, track_ys, track_zs), axis=-1),
            hovertemplate='<b>x</b>: %{customdata[0]:,.2f}<br>' +
                          '<b>y</b>: %{customdata[1]:,.2f}<br>' +
                          '<b>z</b>: %{customdata[2]:,.2f}',
                          showlegend=False)

        traces.append(matched_track_trace)
        
    # If track_ids was not supplied, plot all (unmatched) tracks
    if plot_unmatched_objects:
        for index, track in unmatched_track_df.iterrows():

            track_points = track['points']
            track_id = track['id']

            track_xs = track_points[:,0]
            track_ys = track_points[:,1]
            track_zs = track_points[:,2]

            unmatched_track_trace = go.Scatter3d(
                x = track_xs,
                # Flip y and z so that plotly plots the long axis as the z-direction
                y = track_zs,
                z = track_ys,
                mode='markers',
                marker=dict(size=1, color='lightslategrey'),
                name='Track ID ' + str(track_id),
                customdata=np.stack((track_xs, track_ys, track_zs), axis=-1),
                hovertemplate='<b>x</b>: %{customdata[0]:,.2f}<br>' +
                              '<b>y</b>: %{customdata[1]:,.2f}<br>' +
                              '<b>z</b>: %{customdata[2]:,.2f}',
                              showlegend=False)

            traces.append(unmatched_track_trace)

    ##### CRT HIT TRACES #####
    matched_crthit_id = matched_crthit_df['id']
    matched_crthit_xs = matched_crthit_df['position_x']
    matched_crthit_ys = matched_crthit_df['position_z']
    matched_crthit_zs = matched_crthit_df['position_y']
    matched_crthit_trace = go.Scatter3d(
        x = matched_crthit_xs,
        y = matched_crthit_ys,
        z = matched_crthit_zs,
        mode='markers',
        marker=dict(size=2, color='red'),
        customdata=np.stack((matched_crthit_id, matched_crthit_xs, matched_crthit_ys, matched_crthit_zs), axis=-1),
        #name=[f"CRT Hit ID {id}" for id in crthit_id],
        hovertemplate='<b>ID</b>: %{customdata[0]:i}<br>' +
                      '<b>x</b>: %{customdata[1]:,.2f}<br>' +
                      '<b>y</b>: %{customdata[2]:,.2f}<br>' +
                      '<b>z</b>: %{customdata[3]:,.2f}',
                      showlegend=False)
    traces.append(matched_crthit_trace)
    
    # If track_ids was not supplied, plot all (unmatched) tracks
    if plot_unmatched_objects:
        unmatched_crthit_id = unmatched_crthit_df['id']
        unmatched_crthit_xs = unmatched_crthit_df['position_x']
        unmatched_crthit_ys = unmatched_crthit_df['position_z']
        unmatched_crthit_zs = unmatched_crthit_df['position_y']
        unmatched_crthit_trace = go.Scatter3d(
            x = unmatched_crthit_xs,
            y = unmatched_crthit_ys,
            z = unmatched_crthit_zs,
            mode='markers',
            marker=dict(size=2, color='lightslategrey'),
            customdata=np.stack((unmatched_crthit_id, unmatched_crthit_xs, unmatched_crthit_ys, unmatched_crthit_zs), axis=-1),
            #name=[f"CRT Hit ID {id}" for id in crthit_id],
            hovertemplate='<b>ID</b>: %{customdata[0]:i}<br>' +
                          '<b>x</b>: %{customdata[1]:,.2f}<br>' +
                          '<b>y</b>: %{customdata[2]:,.2f}<br>' +
                          '<b>z</b>: %{customdata[3]:,.2f}',
                          showlegend=False)
        traces.append(unmatched_crthit_trace)

    ##### CRT PLANES #####
    fig = go.Figure(traces)

    for index, row in crt_plane_df.iterrows():
        x_min, x_max, y_min, y_max, z_min, z_max = row[['Xmin (cm)', 'Xmax (cm)', 'Ymin (cm)', 'Ymax (cm)', 'Zmin (cm)', 'Zmax (cm)']]
        plane_region = row['Region Name']
        thin_axis = row['thin range (x=0, y=1, z=2)']
        x_values = [[x_min, x_max], [x_min, x_max]]
        y_values = [[z_min, z_min], [z_max, z_max]]
        z_values = [[y_min, y_min], [y_max, y_max]]
        # Plotly surfaces are weird...have to change the xyz values
        # depending on which axis the plane is aligned with
        if thin_axis == 0:
            x_values = [[x_min, x_min], [x_max, x_max]]
            y_values = [[z_min, z_max], [z_min, z_max]]
            z_values = [[y_min, y_min], [y_max, y_max]]
        plane_trace = go.Surface(
            x=x_values, y=y_values, z=z_values,
            opacity=0.2,
            # A pale green-ish color to distinguish from anode/cathode planes
            colorscale = [[0, 'rgb(224, 255, 255)'], [1, 'rgb(224, 255, 255)']],
            hoverinfo='none',
            name=plane_region
        )
        fig.add_trace(plane_trace)

    surf_x0 = -210.215*np.ones(100)
    surf_x1 = 210.215*np.ones(100)
    surf_y = np.linspace(-181.6, 134.96, 100)
    surf_z = np.linspace(-894.9505, 894.9505, 100)
    surf_ytest = surf_z
    surf_ztest = surf_y
    mycolorscale = [[0, '#aa9ce2'],[1, '#aa9ce2']]

    plane0 = go.Surface(
        x=surf_x0, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x0)), 
        colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='none', name='Plane 0')
    plane1 = go.Surface(
        x=surf_x1, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x1)), 
        colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='none', name='Plane 1')

    fig.add_trace(plane0)
    fig.add_trace(plane1)

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=1.25, z=1.25)
    )

    fig.update_layout(scene = dict(
        xaxis_title='x [cm]',
        yaxis_title='z [cm]',
        zaxis_title='y [cm]'),
        scene_camera=camera,
        height=700, width=900
    )

    fig.update_layout(margin=dict(l=0, r=100, b=0, t=0))
    fig.update_scenes(aspectmode='data')
    iplot(fig)



In [ ]:
plot_unmatched_objects = False
plot_crt_tpc_matches(matched_track_df, unmatched_track_df, 
                     matched_crthit_df, unmatched_crthit_df, 
                     crt_plane_df,
                     matched_track_ids, plot_unmatched_objects)

## 1D Distributions

In [ ]:
matched_crthit_pe = matched_crthit_df['total_pe']
unmatched_crthit_pe = unmatched_crthit_df['total_pe']

#x_upper_range = max(max(matched_crthit_pe), max(unmatched_crthit_pe))
x_upper_range = 500
print(x_upper_range)

plt.hist(unmatched_crthit_pe, bins=20, color='blue', alpha=0.3, label='Unmatched CRT hits', range=(0, x_upper_range))
plt.hist(matched_crthit_pe, bins=20, color='red', alpha=0.3, label='Matched CRT hits', range=(0, x_upper_range))
plt.xlabel('Total PE', fontsize=12)
plt.ylabel('Count/25 PE', fontsize=12)
plt.legend()
plt.show()